# Notebook to create invididual pages for web pages


## Step 1 Create figure on title page
- keep it updated, we can also randomly choose from list of highest cases / deaths 

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import datetime
import os
import random
import pandas as pd
import matplotlib.pyplot as plt

import oscovida as corona

pd.set_option('display.float_format', '{:.2f}'.format)  #  Disable pandas scientific notation
# force download of new data
# clear_cache()



# fetch world data
d, c = corona.fetch_deaths(), corona.fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

def save_frontpage_figure(country):
    fig, ax =  plt.subplots(figsize=(12, 2.5))
    cases, deaths, region_label = corona.get_country_data(country)
    corona.plot_daily_change(ax, cases, 'C1', labels=(region_label, "cases"))
    # ticks and labels on the right y-axis as well
    ax.yaxis.set_ticks_position('both')
    ax.tick_params(left=True, right=True, labelleft=True, labelright=True)
    
    filename = 'fig-homepage-example-plot'
    outdir = 'pelican/content/pages'
    outpath = os.path.join(outdir, filename)
    
    # create fake point for extra text in legend
    xvalue = (ax.get_xlim()[0] + ax.get_xlim()[1])*0.5
    yvalue = (ax.get_ylim()[0] + ax.get_ylim()[1])*0.5
    date_string = datetime.datetime.now().strftime("%d/%m/%Y")  # %H:%M") + " CET"
    extra = ax.plot([xvalue], [yvalue], 'o', alpha=0.001, 
                    label=f"[last updated: {date_string}]")
    
    # change order of labels in legend
    # (see https://stackoverflow.com/a/46160465)
    handles, labels = ax.get_legend_handles_labels()
    order = [2, 0, 1]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])
    
    plt.tight_layout()
    fig.savefig(outpath + '.svg')
    
    # also create html file to go with this
    # Needs to look something like:
    # "<a href="html/US.html"><img src='{attach}fig-homepage-example-plot.svg'></a>"
    # fig.savefig(outpath + '.png', dpi=250)
    with open("pelican/content/pages/index-included-for-figure1-html", "tw") as f:
        f.write(f'<a href="html/{country.replace(" ", "-")}.html">')
        f.write('<img class="noZoom img-responsive" src="{attach}fig-homepage-example-plot.svg"></a>')

    return ax

# save_frontpage_figure('United Kingdom')

In [ ]:
def find_countries_with_highest_cases_deaths(n=5):
    d, c = corona.fetch_deaths(), corona.fetch_cases()
    # most deaths:
    d2 = d.iloc[:, -1]    # last column is most recent date
    c2 = c.iloc[:, -1]
    top_n_deaths = d2.sort_values(ascending=False).head(n)
    top_n_cases = c2.sort_values(ascending=False).head(n)
    
    return top_n_cases, top_n_deaths

def pick_random_country(n=10):
    # use infections
    cases, deaths = find_countries_with_highest_cases_deaths(n=n)
    return random.choice(cases.index)
    

In [ ]:
print(list(find_countries_with_highest_cases_deaths(n=10)[0].index))

todays_country = pick_random_country(n=10)
print(f"Chosen country is {todays_country}")
save_frontpage_figure(todays_country)

In [ ]:
# override if desired:
# save_frontpage_figure("United Kingdom")
# save_frontpage_figure("Brazil")

# ax = save_frontpage_figure("France")

## Step 2: Update data for explanation of plots

This is for https://oscovida.github.io/plots.html

In [ ]:
def save_figure(function_to_call, filename, data):
    fig, ax =  plt.subplots(figsize=(10, 2.5))
    cases, deaths = corona.get_country_data(country)
    corona.plot_daily_change(ax, cases, 'C1')
    filename = 'fig-homepage-example-plot'
    outdir = 'pelican/content/pages'
    outpath = os.path.join(outdir, filename)
    
    # create fake point for extra text in legend
    xvalue = (ax.get_xlim()[0] + ax.get_xlim()[1])*0.5
    yvalue = (ax.get_ylim()[0] + ax.get_ylim()[1])*0.5
    date_string = datetime.datetime.now().strftime("%d/%m/%Y")  # %H:%M") + " CET"
    extra = ax.plot([xvalue], [yvalue], 'o', alpha=0.001, 
                    label=f"[last updated: {date_string}]")
    
    # change order of labels in legend
    # (see https://stackoverflow.com/a/46160465)
    handles, labels = ax.get_legend_handles_labels()
    order = [2, 0, 1]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])
    
    plt.tight_layout()
    fig.savefig(outpath + '.svg')
    
    # also create html file to go with this
    # Needs to look something like:
    # "<a href="html/US.html"><img src='{attach}fig-homepage-example-plot.svg'></a>"
    # fig.savefig(outpath + '.png', dpi=250)
    with open("pelican/content/pages/index-included-for-figure1-html", "tw") as f:
        f.write(f'<a href="html/{country.replace(" ", "-")}.html">')
        f.write('<img class="noZoom img-responsive" src="{attach}fig-homepage-example-plot.svg"></a>')

# save_frontpage_figure('United Kingdom')

In [ ]:
def setup_fig_axes():
    fig, ax =  plt.subplots(figsize=(10, 2.5))
    return fig, ax

def save_axis(fig, filename, path="pelican/content"):
    fig.savefig(os.path.join(path, filename + ".svg"))

def align_axis(template, apply_to):
    """Given 'template' axis object, take some of the settings
    and apply this to another axis setting"""
    # enforce same x-axis on all plots
    apply_to.set_xlim(template.get_xlim())
    apply_to.tick_params(left=True, right=True, labelleft=True, labelright=True)
    apply_to.yaxis.set_ticks_position('both')


In [ ]:
def save_discussion_of_plots(country="Korea, South", region=None, subregion=None):
    c, d, region_label = corona.get_country_data(country, region=region, subregion=subregion)

    basename = "fig-south-korea"
    
    fig, ax1 = setup_fig_axes()
    corona.plot_time_step(ax=ax1, series=c, style="-C1", labels=(region_label, "cases"))
    corona.plot_time_step(ax=ax1, series=d, style="-C0", labels=(region_label, "deaths"))
    align_axis(ax1, ax1)
    save_axis(fig, filename=basename + '1')

    fig, ax = setup_fig_axes()   
    corona.plot_daily_change(ax=ax, series=c, color="C1", labels=(region_label, "cases"))
    align_axis(ax1, ax)
    if country == "China":
        ax.set_ylim(0, 5000)
    save_axis(fig, filename=basename + '2')
    
    fig, ax = setup_fig_axes()   
    corona.plot_daily_change(ax=ax, series=d, color="C0", labels=(region_label, "deaths"))
    align_axis(ax1, ax)
    save_axis(fig, filename=basename + '3')

    fig, ax = setup_fig_axes()   
    # corona.plot_growth_factor(ax, series=d, color="C0")
    # corona.plot_growth_factor(ax, series=c, color="C1")
    corona.plot_reproduction_number(ax, series=c, color_g="C1", color_R="C5", 
                                    labels=(region_label, "cases"))
    align_axis(ax1, ax)
    save_axis(fig, filename=basename + '4')

    fig, ax = setup_fig_axes()   
    corona.plot_reproduction_number(ax, series=d, color_g="C0", color_R="C4", 
                                    labels=(region_label, "deaths"))
    align_axis(ax1, ax)

    save_axis(fig, filename=basename + '5')

    fig, ax = setup_fig_axes()   
    corona.plot_doubling_time(ax, series=d, color="C0", labels=(region_label, "deaths"))
    corona.plot_doubling_time(ax, series=c, color="C1", labels=(region_label, "cases"))
    align_axis(ax1, ax)
    save_axis(fig, filename=basename + '6')
    
    # bad hack: make_compare_plot always draws both axis together (should rework this),
    # we drop one to be able to save the other one alone:
    axes, res_c, res_d = corona.make_compare_plot(country)
    # get rid of title
    axes[0].set_title('')
    fig = axes[0].get_figure()
    # remove second axes
    axes[1].remove()
    save_axis(fig, filename=basename + '7')
    
    # now the other plot
    axes, res_c, res_d = corona.make_compare_plot(country)
    # get rid of title
    fig = axes[0].get_figure()
    # remove first axes
    axes[0].remove()
    save_axis(fig, filename=basename + '8')
    return fig, axes
    

In [ ]:
save_discussion_of_plots()